<h1>Predict Future Sales - Análise exploratória de dados</h1>

   Este Problema de aprendizado de máquina está sendo desenvolvido em Python 3, a competição trabalha com um conjunto de dados de vendas diárias de uma empresa de software da Rússia - <b>1C Company.</b> O objetivo final é prever as vendas totais de cada produto e de cada loja no próximo mês.


<b>Importando Bibliotecas</b>

In [ ]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model, datasets
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from collections import OrderedDict
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from xgboost import XGBRegressor
from xgboost import plot_importance
from xgboost import XGBClassifier
from numpy import loadtxt
from sklearn.metrics import accuracy_score

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

<h2>Entendendo a base de dados</h2>

<h3>Descrição dos arquivos:</h3>
<li><b>sales_train.csv</b> - o conjunto de treinamento. Dados históricos diários de janeiro de 2013 a outubro de 2015.</li>
<li><b>test.csv</b> - o conjunto de teste. O problema se baseia na predição das vendas dessas lojas e de seus produtos para novembro de 2015.</li>
<li><b>sample_submission.csv</b> - um arquivo com o modelo de envio no formato correto.</li>
<li><b>items.csv</b> - informações suplementares sobre os itens / produtos.</li>
<li><b>item_categories.csv</b> - informações suplementares sobre as categorias de itens.</li>
<li><b>shops.csv</b> - informações suplementares sobre as lojas.</li>

<h3>Atributos dos dados:</h3>
<li><b>ID</b> - um identificador que representa uma tupla dentro do conjunto de testes</li>
<li><b>shop_id</b> - identificador único de uma loja </li>
<li><b>item_id</b> - identificador único de um produto </li>
<li><b>item_category_id</b> - identificador único da categoria de um item</li>
<li><b>item_cnt_day</b> - número de produtos vendidos. Será previsto um valor mensal dessa medida</li>
<li><b>item_price</b> - preço atual de um item</li>
<li><b>date</b> - data em formato dia/mês/ano</li>
<li><b>date_block_num</b> - um número consecutivo utilizado para representar um mês. Janeiro de 2013 é 0, fevereiro de 2013 é 1, ..., outubro de 2015 é 33</li>
<li><b>item_name</b> - nome do item</li>
<li><b>shop_name</b> - nome da loja</li>
<li><b>item_category_name</b> - nome da categoria do item</li>





<h2>Lendo os dados</h2>

In [ ]:
f1 = pd.read_csv('../input/sales_train.csv')
f1.head()

<b>Dimensões da tabela</b>

In [ ]:
print(f1.shape)

In [ ]:
f2 = pd.read_csv('../input/items.csv')
f2.head()

In [ ]:
f3 = pd.read_csv('../input/item_categories.csv')
f3.head()

In [ ]:
f4 = pd.read_csv('../input/shops.csv')
f4.head()

In [ ]:
teste = pd.read_csv('../input/test.csv').set_index('ID')
teste

<b>Dimensões das tabelas</b>

In [ ]:
print(f2.shape,f3.shape,f4.shape)

Perceba que existem 22170 itens diferentes, 84 categorias diferentes e 60 lojas diferentes.

<b>Detalhes sobre os preços dos itens: </b>

In [ ]:
f1['item_price'].describe()

Pode-se observar que o maior preço da coluna "item_price" está bem acima da média, tem-se também um valor negativo entre os preços, o que é bem estranho.

<h3>Análises gráficas<h3>

In [ ]:
%matplotlib notebook

f1['item_price'].hist(bins=30,edgecolor='black')

In [ ]:
f1['item_price'].mean()

<b>Pelo gráfico percebemos que a maioria dos itens tem preço abaixo de 25000</b>

In [ ]:
%matplotlib notebook

f1.boxplot(column='item_price')

In [ ]:
%matplotlib notebook

f1.boxplot(column='item_cnt_day')

In [ ]:
f1['item_cnt_day'].mean()

<b>Pelo gráfico no formato boxplot, encontramos 1 valor outlier extremamente longe da média dos preços</b>

In [ ]:
print(f1.count())

<b>Perceba que não existe valores nulos em nenhuma linha do dataset</b>

In [ ]:
%matplotlib notebook
ds1 = f1[f1['item_price'] < 50000]
ds1['item_price'].hist(bins=30,edgecolor='black')

In [ ]:
print(float(ds1['item_price'].shape[0])/float(f1.shape[0]))

Perceba que aproximadamente 99,99990% dos itens tem preços abaixo de 50000

In [ ]:
%matplotlib notebook
ds2 = f1[f1['item_price'] < 5000]
ds2['item_price'].hist(bins=30,edgecolor='black')

In [ ]:
print(float(ds2['item_price'].shape[0])/float(f1.shape[0]))

Perceba que 98,84766% dos itens tem preços abaixo de 5000


<h4>Peneirando a base de dados:</h4>

In [ ]:
dataset=f1[f1['item_price'] < 50000]
f1[f1['item_price'] > 50000]

In [ ]:
dataset['item_price'].mean()

In [ ]:
f1[f1['item_cnt_day'] > 2000]
dataset=f1[f1['item_cnt_day'] < 2000]

Todos os itens com preços acima de 50000 tiveram apenas 1 venda durante todo o periodo, pode-se então considerar estas vendas como eventos isolados e retirar as linhas da base de dados. Em um único dia foi vendida 2000 unidades de um mesmo item, como só existe ocorrência dessa quantidade de vendas em uma única linha, pode-se considerar como outro evento isolado.

In [ ]:
dataset=dataset[dataset['item_price'] > 0]
f1[f1['item_price'] <= 0]

Existe uma única venda registrada com o "item_price" negativo, isso também pode ser considerado um evento isolado.

In [ ]:
devolvidos = f1[f1['item_cnt_day'] <= 0]
f1[f1['item_cnt_day'] <= 0]

Um grande problema da base de dados são os itens com "item_cnt_day" < 0, muito provavelmente estas foram devoluções de itens. Naturalmente, produtos, sejam de qualquer área, podem vir defeituosos ou não suprirem as espectativas do cliente, porém se há uma grande frequência de devolução de um mesmo produto, com o passar do tempo, sua popularidade tenderá a cair.

In [ ]:
%matplotlib notebook
devolvidos['shop_id'].hist(bins=60,edgecolor='black')


In [ ]:
%matplotlib notebook
devolvidos['item_id'].hist(bins=22170,edgecolor='black')

In [ ]:
devolvidos['item_id'].value_counts()


Perceba que um item muito comprado, ja que todo item pode apresentar defeito, pode ser um dos itens mais devolvidos. Portanto a análise de devolução deve ser feita em termo de % e não de quantidade de devoluções, devemos analisar a quantidade de itens devolvidos/quantidade de itens comprados. 

In [ ]:
print(dataset['item_cnt_day'].max(),)

In [ ]:
%matplotlib notebook
plt.scatter(y=dataset['item_cnt_day'], x=dataset['item_price'], color='blue', s=50, alpha=.5)
X_plot = sp.linspace(min(dataset['item_price']), max(dataset['item_price']), len(dataset['item_price']))
plt.ylim(-300,3000)
plt.xlim(-1000,50000)
plt.title('Gráfico de dispersão')
plt.ylabel('$y$ - Quantidade de itens vendidos')
plt.xlabel('$x$ - Preço do item')
plt.show()

In [ ]:
%matplotlib notebook
plt.scatter(y=dataset['item_cnt_day'], x=dataset['item_price'], color='blue', s=50, alpha=.5)
X_plot = sp.linspace(min(dataset['item_price']), max(dataset['item_price']), len(dataset['item_price']))
plt.ylim(-300,3000)
plt.xlim(-300,5000)
plt.title('Gráfico de dispersão')
plt.ylabel('$y$ - Quantidade de itens vendidos de uma só vez')
plt.xlabel('$x$ - Preço do item')
plt.show()

In [ ]:
print((f1[f1['item_price'] <= 5000].shape[0])/f1['item_price'].shape[0])

<b>Correlação entre as variáveis:</b>

In [ ]:
dataset.corr()

In [ ]:
plt.figure(figsize=(7,4)) 
sns.heatmap(dataset.corr(),annot=True,cmap='cubehelix_r')
plt.show()

<b>Em termos de variáveis quantitativas temos apenas 1 correlação válida,  "item_price " ---> "item_cnt_day"  = 0.011</b>


<h3>Popularidade das lojas</h3>

In [ ]:
print(dataset['shop_id'].max())

In [ ]:
%matplotlib notebook
dataset['shop_id'].hist(bins=60,edgecolor='black')

<h3>Vedas mensais por ano</h3>

In [ ]:
ano1=dataset[dataset['date_block_num'] <= 11]
ano2=dataset[dataset['date_block_num'] > 11]
ano2=ano2[ano2['date_block_num'] <= 23]
ano3=dataset[dataset['date_block_num'] > 23]
print("Minimo mês do ano 1:",ano1['date_block_num'].min(),"Máximo mês do ano 1:",ano1['date_block_num'].max())
print("Minimo mês do ano 2:",ano2['date_block_num'].min(),"Máximo mês do ano 2:",ano2['date_block_num'].max())
print("Minimo mês do ano 3:",ano3['date_block_num'].min(),"Máximo mês do ano 3:",ano3['date_block_num'].max())

In [ ]:
%matplotlib notebook
ano1['date_block_num'].hist(bins=12,edgecolor='black',alpha=0.5)

In [ ]:
%matplotlib notebook
ano2['date_block_num'].hist(bins=12,edgecolor='black',alpha=0.5)

In [ ]:
%matplotlib notebook
ano3['date_block_num'].hist(bins=10,edgecolor='black',alpha=0.5)

In [ ]:
dataset=pd.merge(dataset, f2, how='inner')
dataset.sort_values(by=['date'], inplace=True)
dataset.head(3)

In [ ]:
print(dataset['item_category_id'].max())

In [ ]:
%matplotlib notebook
dataset['item_category_id'].hist(bins=84,edgecolor='black')

In [ ]:
'''
v=[]
w=[]
x=[]
y=[]
z=[]
for i in range (33):
    filtroMes = dataset[dataset['date_block_num'] == i]
    for j in range (60):
        filtroShop = filtroMes[filtroMes['shop_id'] == j]
        for k in range (22170):
            filtroItem = filtroShop[filtroShop['item_id'] == k]
            if(filtroItem.count()[1] > 0):
                v.append(filtroItem.iloc[0][1])
                w.append(filtroItem.iloc[0][2])
                x.append(filtroItem.iloc[0][3])
                y.append(filtroItem['item_price'].mean())
                z.append(filtroItem['item_cnt_day'].sum())


#data = OrderedDict(
#{
#'date_block_num': [teste.iloc[0][1],teste.iloc[0][1]],
#'shop_id': [teste.iloc[0][2],teste.iloc[0][2]],
#'item_id': [teste.iloc[0][3],teste.iloc[0][3]],
#'item_price': [teste['item_price'].mean(),teste.iloc[0][4]],
#'item_cnt_month': [teste['item_cnt_day'].sum(),teste.iloc[0][5]]
#})

data = OrderedDict(
{
'date_block_num': v,
'shop_id': w,
'item_id': x,
'item_price': y,
'item_cnt_month': z
})
df = pd.DataFrame(data)
df.to_csv('filtro1.csv')
df
'''

<h2>Features</h2>

Antes de minerarmos possíveis caracteristicas para os dados, vamos sintetizar todo o aprendizado da análise:
<li>A maior parte dos produtos vendidos tem preço abaixo de 5000</li> 
<li>A partir de um certo ponto, itens mais caros tendem a ser menos vendidos</li> 
<li>Durante 2 anos consecutivos o mês de dezembro foi o mês que houve o maior índice de vendas, provavelmente por conta do natal</li>
<li>Os gráficos de vendas mesais por ano tem o comportamento de um vale, maiores nas extremidades e menores no centro </li>
<li>Algumas lojas sã resposáveis por quase todas as vendas

Antes de criar as fetures vamos separar a base de dados em conjuntos únicos de shop/item para cada mês, dessa forma trabalharemos de forma similar ao conjuto de testes. Também limitaremos os valores de 'item_cnt_month' entre 0 e 20 ( como recomendado pelos moderadores da competição).

In [ ]:
matrix = []
cols = ['date_block_num','shop_id','item_id']

for i in range(34):
    sales = dataset[dataset.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)


group = dataset.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20)
                                .astype(np.float16))
matrix

Anexando o conjunto de teste ao dataset como o mês 34, isso facilitará na hora de criar as features.

In [ ]:
teste['date_block_num'] = 34
teste['date_block_num'] = teste['date_block_num'].astype(np.int8)
teste['shop_id'] = teste['shop_id'].astype(np.int8)
teste['item_id'] = teste['item_id'].astype(np.int16)
matrix = pd.concat([matrix, teste], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)

Agora serão adicionadas 2 features importantes para séries temporais, o mês e o dia (perceba que os números simbolizam variáveis categóricas).

In [ ]:
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)

A proxima feature é quatos meses desde a útima venda daquele item naquela loja

In [ ]:
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num    
matrix

In [ ]:
'''matrix['item_shop_last_sale'].describe()

A proxima feature é quatos meses desde a útima venda daquele item.

In [ ]:
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num   
            
matrix

In [ ]:
matrix=pd.merge(matrix, f2, how='inner')
matrix=pd.merge(matrix, f1, how='inner')
matrix


In [ ]:
'''matrix['item_category_id'].describe()

In [ ]:
'''matrix['item_last_sale'].describe()

Fução para criar as lag_features
****

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

Criando 4 novas lag features, baseadas na quantidade dos itens vendidos no mês, para o ultimo mês, os ultimos 2 meses, os ultimos 3 meses e os ultimos 6 meses


In [ ]:
'''matrix = lag_feature(matrix, [1,2,3,6], 'item_cnt_month')
matrix

Criando as lag_features para: 


Média de itens vendidos no ultimo mês


In [ ]:
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)
matrix

Média dos itens vendidos nos ultimos meses, por itens.

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
matrix

Média de itens vendidos no mês passado, por lojas.

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)
matrix

Média dos itens vendidos no mês passado, por categorias

In [ ]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)
matrix'''


In [ ]:
group = dataset.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = dataset.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

# https://stackoverflow.com/questions/31828240/first-non-null-value-per-row-from-a-list-of-pandas-columns/31828559
# matrix['price_trend'] = matrix[['delta_price_lag_1','delta_price_lag_2','delta_price_lag_3']].bfill(axis=1).iloc[:, 0]
# Invalid dtype for backfill_2d [float16]

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)



Média dos preços dos itens vendidos nos ultimos meses.

In [ ]:
'''group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,4,5,6], 'item_avg_item_price')
matrix.drop(['item_avg_item_price'], axis=1, inplace=True)'''

In [ ]:
'''group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
'''matrix

Agora, todas as lag_features serão percorridas para que os valores nulos sejam corrigidos.


In [ ]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)
matrix

Drop dos 6 primeiros meses e da coluna de nomes de itens

In [ ]:
matrix = matrix[matrix.date_block_num > 5]
columns=['item_name','item_category_name','date','delta_price_lag']
for c in columns:
    if c in matrix:
        matrix.drop(c, axis = 1, inplace = True)
matrix

In [ ]:
'''matrix['date_item_avg_item_cnt_lag_1'].describe()

<h2>Aplicando xgboost</h2>

In [ ]:
X_train = matrix[matrix.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = matrix[matrix.date_block_num < 33]['item_cnt_month']
X_valid = matrix[matrix.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = matrix[matrix.date_block_num == 33]['item_cnt_month']
X_test = matrix[matrix.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=5,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 4)
    plot_importance(model)
    pyplot.show()

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({
    "ID": teste.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)
matrix.to_csv('data.csv', index=False)
del matrix
plot_importance(model)
pyplot.show()